<a href="https://colab.research.google.com/github/Mrutyunjay01/Employee_attrition_rate_prediction_challenge_hackerearth2020/blob/master/NN_with_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [ ]:
train_df = pd.read_csv('/content/Train.csv')
train_df.head()

,Employee_ID,Gender,Age,Education_Level,Relationship_Status,Hometown,Unit,Decision_skill_possess,Time_of_service,Time_since_promotion,growth_rate,Travel_Rate,Post_Level,Pay_Scale,Compensation_and_Benefits,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,Attrition_rate
0,EID_23371,F,42.0,4,Married,Franklin,IT,Conceptual,4.0,4,33,1,1,7.0,type2,3.0,4,0.7516,1.8688,2.0,4,5,3,0.1841
1,EID_18000,M,24.0,3,Single,Springfield,Logistics,Analytical,5.0,4,36,0,3,6.0,type2,4.0,3,-0.9612,-0.4537,2.0,3,5,3,0.0670
2,EID_3891,F,58.0,3,Married,Clinton,Quality,Conceptual,27.0,3,51,0,2,8.0,type2,1.0,4,-0.9612,-0.4537,3.0,3,8,3,0.0851
3,EID_17492,F,26.0,3,Single,Lebanon,Human Resource Management,Behavioral,4.0,3,56,1,3,8.0,type2,1.0,3,-1.8176,-0.4537,NaN,3,7,3,0.0668
4,EID_22534,F,31.0,1,Married,Springfield,Logistics,Conceptual,5.0,4,62,1,3,2.0,type3,3.0,1,0.7516,-0.4537,2.0,2,8,2,0.1827


In [ ]:
categorical_train = train_df.select_dtypes('object')
categorical_train = categorical_train.drop(columns=['Employee_ID'])
categorical_train

,Gender,Relationship_Status,Hometown,Unit,Decision_skill_possess,Compensation_and_Benefits
0,F,Married,Franklin,IT,Conceptual,type2
1,M,Single,Springfield,Logistics,Analytical,type2
2,F,Married,Clinton,Quality,Conceptual,type2
3,F,Single,Lebanon,Human Resource Management,Behavioral,type2
4,F,Married,Springfield,Logistics,Conceptual,type3
...,...,...,...,...,...,...
6995,F,Married,Franklin,Operarions,Behavioral,type3
6996,F,Married,Lebanon,R&D,Analytical,type3
6997,F,Single,Springfield,IT,Directive,type2
6998,F,Married,Washington,Sales,Behavioral,type0


In [ ]:
categorical_train.isna().sum()

Employee_ID                  0
Gender                       0
Relationship_Status          0
Hometown                     0
Unit                         0
Decision_skill_possess       0
Compensation_and_Benefits    0
dtype: int64

In [ ]:
categorical_train_dummies = pd.get_dummies(categorical_train.drop(columns=['Employee_ID']))
categorical_train_dummies

,Gender_F,Gender_M,Relationship_Status_Married,Relationship_Status_Single,Hometown_Clinton,Hometown_Franklin,Hometown_Lebanon,Hometown_Springfield,Hometown_Washington,Unit_Accounting and Finance,Unit_Human Resource Management,Unit_IT,Unit_Logistics,Unit_Marketing,Unit_Operarions,Unit_Production,Unit_Purchasing,Unit_Quality,Unit_R&D,Unit_Sales,Unit_Security,Decision_skill_possess_Analytical,Decision_skill_possess_Behavioral,Decision_skill_possess_Conceptual,Decision_skill_possess_Directive,Compensation_and_Benefits_type0,Compensation_and_Benefits_type1,Compensation_and_Benefits_type2,Compensation_and_Benefits_type3,Compensation_and_Benefits_type4
0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
1,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
2,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0
3,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
4,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
6996,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0
6997,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
6998,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0


In [ ]:
numerical_train = train_df.select_dtypes(exclude='object')
numerical_train

,Age,Education_Level,Time_of_service,Time_since_promotion,growth_rate,Travel_Rate,Post_Level,Pay_Scale,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,Attrition_rate
0,42.0,4,4.0,4,33,1,1,7.0,3.0,4,0.7516,1.8688,2.0,4,5,3,0.1841
1,24.0,3,5.0,4,36,0,3,6.0,4.0,3,-0.9612,-0.4537,2.0,3,5,3,0.0670
2,58.0,3,27.0,3,51,0,2,8.0,1.0,4,-0.9612,-0.4537,3.0,3,8,3,0.0851
3,26.0,3,4.0,3,56,1,3,8.0,1.0,3,-1.8176,-0.4537,NaN,3,7,3,0.0668
4,31.0,1,5.0,4,62,1,3,2.0,3.0,1,0.7516,-0.4537,2.0,2,8,2,0.1827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,23.0,5,4.0,1,20,1,2,6.0,3.0,1,0.7516,-0.4537,2.0,3,7,3,0.7839
6996,44.0,1,16.0,2,59,0,2,4.0,3.0,2,-0.1048,-2.7762,2.0,4,8,4,0.1055
6997,49.0,3,10.0,1,59,1,2,3.0,4.0,2,-0.1048,-0.4537,2.0,4,8,3,0.7847
6998,47.0,3,24.0,1,21,1,5,8.0,2.0,4,0.7516,1.8688,2.0,3,6,3,0.4162


In [ ]:
to_clean_train = pd.concat([numerical_train, categorical_train_dummies], axis=1)
to_clean_train

,Age,Education_Level,Time_of_service,Time_since_promotion,growth_rate,Travel_Rate,Post_Level,Pay_Scale,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,Attrition_rate,Gender_F,Gender_M,Relationship_Status_Married,Relationship_Status_Single,Hometown_Clinton,Hometown_Franklin,Hometown_Lebanon,Hometown_Springfield,Hometown_Washington,Unit_Accounting and Finance,Unit_Human Resource Management,Unit_IT,Unit_Logistics,Unit_Marketing,Unit_Operarions,Unit_Production,Unit_Purchasing,Unit_Quality,Unit_R&D,Unit_Sales,Unit_Security,Decision_skill_possess_Analytical,Decision_skill_possess_Behavioral,Decision_skill_possess_Conceptual,Decision_skill_possess_Directive,Compensation_and_Benefits_type0,Compensation_and_Benefits_type1,Compensation_and_Benefits_type2,Compensation_and_Benefits_type3,Compensation_and_Benefits_type4
0,42.0,4,4.0,4,33,1,1,7.0,3.0,4,0.7516,1.8688,2.0,4,5,3,0.1841,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
1,24.0,3,5.0,4,36,0,3,6.0,4.0,3,-0.9612,-0.4537,2.0,3,5,3,0.0670,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
2,58.0,3,27.0,3,51,0,2,8.0,1.0,4,-0.9612,-0.4537,3.0,3,8,3,0.0851,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0
3,26.0,3,4.0,3,56,1,3,8.0,1.0,3,-1.8176,-0.4537,NaN,3,7,3,0.0668,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
4,31.0,1,5.0,4,62,1,3,2.0,3.0,1,0.7516,-0.4537,2.0,2,8,2,0.1827,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,23.0,5,4.0,1,20,1,2,6.0,3.0,1,0.7516,-0.4537,2.0,3,7,3,0.7839,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
6996,44.0,1,16.0,2,59,0,2,4.0,3.0,2,-0.1048,-2.7762,2.0,4,8,4,0.1055,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0
6997,49.0,3,10.0,1,59,1,2,3.0,4.0,2,-0.1048,-0.4537,2.0,4,8,3,0.7847,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
6998,47.0,3,24.0,1,21,1,5,8.0,2.0,4,0.7516,1.8688,2.0,3,6,3,0.4162,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0


In [ ]:
abs(to_clean_train.corr()['Attrition_rate']).sort_values(ascending=False)

Attrition_rate                       1.000000
Gender_M                             0.028544
Gender_F                             0.028544
Decision_skill_possess_Behavioral    0.027524
Compensation_and_Benefits_type0      0.026905
VAR2                                 0.023991
Unit_IT                              0.022529
Work_Life_balance                    0.020746
Compensation_and_Benefits_type3      0.017876
Decision_skill_possess_Conceptual    0.016568
Time_of_service                      0.016447
Post_Level                           0.016402
Age                                  0.015498
VAR7                                 0.015299
Unit_Purchasing                      0.015279
Pay_Scale                            0.015236
Unit_Operarions                      0.015147
Hometown_Springfield                 0.014555
growth_rate                          0.014247
Time_since_promotion                 0.013880
Unit_R&D                             0.013868
Decision_skill_possess_Directive  

In [ ]:
for col in abs(to_clean_train.corr()['Attrition_rate']).sort_values(ascending=False).index.to_list():
  if abs(to_clean_train.corr()['Attrition_rate']).sort_values(ascending=False)[col] < 0.015:
    to_clean_train = to_clean_train.drop(columns=col)

to_clean_train

,Age,Time_of_service,Post_Level,Pay_Scale,Work_Life_balance,VAR2,VAR7,Attrition_rate,Gender_F,Gender_M,Unit_IT,Unit_Operarions,Unit_Purchasing,Decision_skill_possess_Behavioral,Decision_skill_possess_Conceptual,Compensation_and_Benefits_type0,Compensation_and_Benefits_type3
0,42.0,4.0,1,7.0,3.0,0.7516,3,0.1841,1,0,1,0,0,0,1,0,0
1,24.0,5.0,3,6.0,4.0,-0.9612,3,0.0670,0,1,0,0,0,0,0,0,0
2,58.0,27.0,2,8.0,1.0,-0.9612,3,0.0851,1,0,0,0,0,0,1,0,0
3,26.0,4.0,3,8.0,1.0,-1.8176,3,0.0668,1,0,0,0,0,1,0,0,0
4,31.0,5.0,3,2.0,3.0,0.7516,2,0.1827,1,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,23.0,4.0,2,6.0,3.0,0.7516,3,0.7839,1,0,0,1,0,1,0,0,1
6996,44.0,16.0,2,4.0,3.0,-0.1048,4,0.1055,1,0,0,0,0,0,0,0,1
6997,49.0,10.0,2,3.0,4.0,-0.1048,3,0.7847,1,0,1,0,0,0,0,0,0
6998,47.0,24.0,5,8.0,2.0,0.7516,3,0.4162,1,0,0,0,0,1,0,1,0


In [ ]:
to_clean_train.isna().sum()

Age                                  412
Time_of_service                      144
Post_Level                             0
Pay_Scale                              9
Work_Life_balance                     11
VAR2                                 577
VAR7                                   0
Attrition_rate                         0
Gender_F                               0
Gender_M                               0
Unit_IT                                0
Unit_Operarions                        0
Unit_Purchasing                        0
Decision_skill_possess_Behavioral      0
Decision_skill_possess_Conceptual      0
Compensation_and_Benefits_type0        0
Compensation_and_Benefits_type3        0
dtype: int64

In [ ]:
to_clean_train = to_clean_train.fillna(0)

In [ ]:
to_clean_train.corr()['Attrition_rate'].sort_values(ascending=False)

Attrition_rate                       1.000000
Gender_M                             0.028544
Compensation_and_Benefits_type0      0.026905
Work_Life_balance                    0.021115
Decision_skill_possess_Conceptual    0.016568
Post_Level                           0.016402
Unit_Purchasing                      0.015279
Unit_Operarions                      0.015147
Age                                 -0.008505
VAR7                                -0.015299
Pay_Scale                           -0.015643
Compensation_and_Benefits_type3     -0.017876
Time_of_service                     -0.019863
Unit_IT                             -0.022529
VAR2                                -0.022906
Decision_skill_possess_Behavioral   -0.027524
Gender_F                            -0.028544
Name: Attrition_rate, dtype: float64

In [ ]:
final_features = to_clean_train.drop(['Age'], axis=1)

In [ ]:
final_features

,Time_of_service,Post_Level,Pay_Scale,Work_Life_balance,VAR2,VAR7,Attrition_rate,Gender_F,Gender_M,Unit_IT,Unit_Operarions,Unit_Purchasing,Decision_skill_possess_Behavioral,Decision_skill_possess_Conceptual,Compensation_and_Benefits_type0,Compensation_and_Benefits_type3
0,4.0,1,7.0,3.0,0.7516,3,0.1841,1,0,1,0,0,0,1,0,0
1,5.0,3,6.0,4.0,-0.9612,3,0.0670,0,1,0,0,0,0,0,0,0
2,27.0,2,8.0,1.0,-0.9612,3,0.0851,1,0,0,0,0,0,1,0,0
3,4.0,3,8.0,1.0,-1.8176,3,0.0668,1,0,0,0,0,1,0,0,0
4,5.0,3,2.0,3.0,0.7516,2,0.1827,1,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,4.0,2,6.0,3.0,0.7516,3,0.7839,1,0,0,1,0,1,0,0,1
6996,16.0,2,4.0,3.0,-0.1048,4,0.1055,1,0,0,0,0,0,0,0,1
6997,10.0,2,3.0,4.0,-0.1048,3,0.7847,1,0,1,0,0,0,0,0,0
6998,24.0,5,8.0,2.0,0.7516,3,0.4162,1,0,0,0,0,1,0,1,0


In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(final_features.drop(columns=['Attrition_rate']), final_features['Attrition_rate'],
                                                    test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler

x_train_s = StandardScaler().fit_transform(x_train)
x_test_s = StandardScaler().fit_transform(x_test)

In [ ]:
final_features

,Time_of_service,Post_Level,Pay_Scale,Work_Life_balance,VAR2,VAR7,Attrition_rate,Gender_F,Gender_M,Unit_IT,Unit_Operarions,Unit_Purchasing,Decision_skill_possess_Behavioral,Decision_skill_possess_Conceptual,Compensation_and_Benefits_type0,Compensation_and_Benefits_type3
0,4.0,1,7.0,3.0,0.7516,3,0.1841,1,0,1,0,0,0,1,0,0
1,5.0,3,6.0,4.0,-0.9612,3,0.0670,0,1,0,0,0,0,0,0,0
2,27.0,2,8.0,1.0,-0.9612,3,0.0851,1,0,0,0,0,0,1,0,0
3,4.0,3,8.0,1.0,-1.8176,3,0.0668,1,0,0,0,0,1,0,0,0
4,5.0,3,2.0,3.0,0.7516,2,0.1827,1,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,4.0,2,6.0,3.0,0.7516,3,0.7839,1,0,0,1,0,1,0,0,1
6996,16.0,2,4.0,3.0,-0.1048,4,0.1055,1,0,0,0,0,0,0,0,1
6997,10.0,2,3.0,4.0,-0.1048,3,0.7847,1,0,1,0,0,0,0,0,0
6998,24.0,5,8.0,2.0,0.7516,3,0.4162,1,0,0,0,0,1,0,1,0


In [ ]:
dtrain = xgb.DMatrix(final_features.drop(['Attrition_rate'], axis=1), final_features['Attrition_rate'])

import warnings
warnings.simplefilter('ignore')

In [ ]:
!pip install bayesian-optimization==1.0.0

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.0.0-cp36-none-any.whl size=10270 sha256=2aaffc0bca7fd0d1e105a2b464220b44f3a5c74ff8a667117c2d033f8fb57abf
  Stored in directory: /root/.cache/pip/wheels/4b/60/a2/6eec4adc73341c1a2a653022b99ff8e1cb090684dba83be683
Successfully built bayesian-optimization


In [ ]:
from bayes_opt import BayesianOptimization

In [ ]:
#Bayesian Optimization function for xgboost
#specify the parameters you want to tune as keyword arguments
def bo_tune_xgb(max_depth, gamma,learning_rate):
    
    params = {'max_depth': int(max_depth),
              'gamma': gamma,
              'learning_rate':learning_rate,
              'subsample': 0.8,
              'eta': 0.1,
              'eval_metric': 'rmse'}
    #Cross validating with the specified parameters in 5 folds and 70 iterations
    cv_result = xgb.cv(params, dtrain, num_boost_round=70, nfold=5)
    #Return the negative RMSE
    return -1.0 * cv_result['test-rmse-mean'].iloc[-1]

#Invoking the Bayesian Optimizer with the specified parameters to tune
xgb_bo = BayesianOptimization(bo_tune_xgb, {'max_depth': (3, 10),
                                             'gamma': (0, 1),
                                             'learning_rate':(0,1),
                                            })

#performing Bayesian optimization for 5 iterations with 8 steps of random exploration with an #acquisition function of expected improvement
xgb_bo.maximize(n_iter=5, init_points=8, acq='ei')

|   iter    |  target   |   gamma   | learni... | max_depth |
-------------------------------------------------------------
|  1        | -0.1953   |  0.3836   |  0.8077   |  7.49     |
|  2        | -0.187    |  0.8464   |  0.6661   |  9.13     |
|  3        | -0.1871   |  0.3388   |  0.1596   |  7.49     |
|  4        | -0.1862   |  0.4544   |  0.1392   |  6.146    |
|  5        | -0.1865   |  0.01106  |  0.1008   |  3.033    |
|  6        | -0.1857   |  0.9314   |  0.1258   |  4.815    |
|  7        | -0.2293   |  0.05337  |  0.8458   |  6.155    |
|  8        | -0.1864   |  0.1534   |  0.04451  |  4.878    |
|  9        | -0.3619   |  1.0      |  0.0      |  7.125    |
|  10       | -0.2749   |  0.0      |  1.0      |  10.0     |
|  11       | -0.1858   |  0.9905   |  0.05545  |  3.135    |
|  12       | -0.2227   |  0.0      |  0.395    |  8.426    |
|  13       | -0.1863   |  1.0      |  1.0      |  10.0     |


In [ ]:
params = xgb_bo.max['params']
print(params)
params['max_depth'] = int(params['max_depth'])

{'gamma': 0.931399542698504, 'learning_rate': 0.12579767830775979, 'max_depth': 4.815025511906325}


In [ ]:
xgbreg = xgb.XGBRegressor(**params).fit(x_train_s, y_train)

[18:13:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
xgbreg.predict(x_test_s)

array([0.1897563, 0.1897563, 0.1897563, ..., 0.1897563, 0.1897563,
       0.1897563], dtype=float32)

In [ ]:
from sklearn.metrics import mean_squared_error

1 - np.sqrt(mean_squared_error(y_test, xgbreg.predict(x_test_s)))

0.8134180946217047

In [ ]:
train_df.Attrition_rate.mean()

0.1893761285714282

In [ ]:
test_df = pd.read_csv('/content/Test.csv')

In [ ]:
test_df_pre = test_df.drop(['Employee_ID'], axis=1)

In [ ]:
num_test = test_df_pre.select_dtypes(exclude='object')

In [ ]:
cat_test = test_df_pre.select_dtypes(include='object')

In [ ]:
cat_test_dummies = pd.get_dummies(cat_test)
cat_test_dummies

,Gender_F,Gender_M,Relationship_Status_Married,Relationship_Status_Single,Hometown_Clinton,Hometown_Franklin,Hometown_Lebanon,Hometown_Springfield,Hometown_Washington,Unit_Accounting and Finance,Unit_Human Resource Management,Unit_IT,Unit_Logistics,Unit_Marketing,Unit_Operarions,Unit_Production,Unit_Purchasing,Unit_Quality,Unit_R&D,Unit_Sales,Unit_Security,Decision_skill_possess_Analytical,Decision_skill_possess_Behavioral,Decision_skill_possess_Conceptual,Decision_skill_possess_Directive,Compensation_and_Benefits_type0,Compensation_and_Benefits_type1,Compensation_and_Benefits_type2,Compensation_and_Benefits_type3,Compensation_and_Benefits_type4
0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0
1,0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
2,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0
3,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
4,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0
2996,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
2997,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
2998,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0


In [ ]:
num_test.isna().sum()

Age                     161
Education_Level           0
Time_of_service          52
Time_since_promotion      0
growth_rate               0
Travel_Rate               0
Post_Level                0
Pay_Scale                 3
Work_Life_balance         5
VAR1                      0
VAR2                    217
VAR3                      0
VAR4                    298
VAR5                      0
VAR6                      0
VAR7                      0
dtype: int64

In [ ]:
final_features.columns

Index(['Time_of_service', 'Post_Level', 'Pay_Scale', 'Work_Life_balance',
       'VAR2', 'VAR7', 'Attrition_rate', 'Gender_F', 'Gender_M', 'Unit_IT',
       'Unit_Operarions', 'Unit_Purchasing',
       'Decision_skill_possess_Behavioral',
       'Decision_skill_possess_Conceptual', 'Compensation_and_Benefits_type0',
       'Compensation_and_Benefits_type3'],
      dtype='object')

In [ ]:
to_clean_test = pd.concat([num_test, cat_test_dummies], axis=1)
to_clean_test

,Age,Education_Level,Time_of_service,Time_since_promotion,growth_rate,Travel_Rate,Post_Level,Pay_Scale,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,Gender_F,Gender_M,Relationship_Status_Married,Relationship_Status_Single,Hometown_Clinton,Hometown_Franklin,Hometown_Lebanon,Hometown_Springfield,Hometown_Washington,Unit_Accounting and Finance,Unit_Human Resource Management,Unit_IT,Unit_Logistics,Unit_Marketing,Unit_Operarions,Unit_Production,Unit_Purchasing,Unit_Quality,Unit_R&D,Unit_Sales,Unit_Security,Decision_skill_possess_Analytical,Decision_skill_possess_Behavioral,Decision_skill_possess_Conceptual,Decision_skill_possess_Directive,Compensation_and_Benefits_type0,Compensation_and_Benefits_type1,Compensation_and_Benefits_type2,Compensation_and_Benefits_type3,Compensation_and_Benefits_type4
0,32.0,5,7.0,4,30,1,5,4.0,1.0,3,-0.9612,-0.4537,2.0,1,8,4,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0
1,65.0,2,41.0,2,72,1,1,1.0,1.0,4,-0.9612,0.7075,1.0,2,8,2,0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
2,52.0,3,21.0,3,25,0,1,8.0,1.0,4,-0.1048,0.7075,2.0,1,9,3,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0
3,50.0,5,11.0,4,28,1,1,2.0,4.0,3,-0.1048,0.7075,2.0,2,8,3,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
4,44.0,3,12.0,4,47,1,3,2.0,4.0,4,1.6081,0.7075,2.0,2,7,4,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,32.0,3,3.0,3,38,1,3,6.0,2.0,1,-0.1048,0.7075,1.0,4,7,4,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0
2996,NaN,2,10.0,2,57,1,4,9.0,1.0,3,0.7516,0.7075,3.0,4,8,3,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
2997,60.0,4,35.0,1,69,0,2,9.0,3.0,3,-1.8176,-0.4537,NaN,4,8,4,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
2998,51.0,1,23.0,1,30,0,2,6.0,1.0,5,0.7516,-0.4537,2.0,3,9,2,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0


In [ ]:
for col in to_clean_test.columns:
  if col not in final_features.columns:
    to_clean_test = to_clean_test.drop(columns=col)

to_clean_test

,Time_of_service,Post_Level,Pay_Scale,Work_Life_balance,VAR2,VAR7,Gender_F,Gender_M,Unit_IT,Unit_Operarions,Unit_Purchasing,Decision_skill_possess_Behavioral,Decision_skill_possess_Conceptual,Compensation_and_Benefits_type0,Compensation_and_Benefits_type3
0,7.0,5,4.0,1.0,-0.9612,4,1,0,0,0,0,0,1,0,0
1,41.0,1,1.0,1.0,-0.9612,2,0,1,1,0,0,0,0,0,0
2,21.0,1,8.0,1.0,-0.1048,3,0,1,0,0,0,0,0,0,1
3,11.0,1,2.0,4.0,-0.1048,3,0,1,0,0,0,0,0,1,0
4,12.0,3,2.0,4.0,1.6081,4,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,3.0,3,6.0,2.0,-0.1048,4,1,0,0,0,0,0,0,0,0
2996,10.0,4,9.0,1.0,0.7516,3,1,0,0,0,0,0,1,0,0
2997,35.0,2,9.0,3.0,-1.8176,4,1,0,0,0,0,0,0,0,1
2998,23.0,2,6.0,1.0,0.7516,2,1,0,1,0,0,1,0,0,1


In [ ]:
final_features

,Time_of_service,Time_since_promotion,growth_rate,Travel_Rate,Post_Level,Pay_Scale,Work_Life_balance,VAR2,VAR7,Attrition_rate,Gender_F,Gender_M,Hometown_Springfield,Unit_IT,Unit_Logistics,Unit_Operarions,Unit_Purchasing,Unit_R&D,Decision_skill_possess_Behavioral,Decision_skill_possess_Conceptual,Decision_skill_possess_Directive,Compensation_and_Benefits_type0,Compensation_and_Benefits_type1,Compensation_and_Benefits_type3
0,4.0,4,33,1,1,7.0,3.0,0.7516,3,0.1841,1,0,0,1,0,0,0,0,0,1,0,0,0,0
1,5.0,4,36,0,3,6.0,4.0,-0.9612,3,0.0670,0,1,1,0,1,0,0,0,0,0,0,0,0,0
2,27.0,3,51,0,2,8.0,1.0,-0.9612,3,0.0851,1,0,0,0,0,0,0,0,0,1,0,0,0,0
3,4.0,3,56,1,3,8.0,1.0,-1.8176,3,0.0668,1,0,0,0,0,0,0,0,1,0,0,0,0,0
4,5.0,4,62,1,3,2.0,3.0,0.7516,2,0.1827,1,0,1,0,1,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,4.0,1,20,1,2,6.0,3.0,0.7516,3,0.7839,1,0,0,0,0,1,0,0,1,0,0,0,0,1
6996,16.0,2,59,0,2,4.0,3.0,-0.1048,4,0.1055,1,0,0,0,0,0,0,1,0,0,0,0,0,1
6997,10.0,1,59,1,2,3.0,4.0,-0.1048,3,0.7847,1,0,1,1,0,0,0,0,0,0,1,0,0,0
6998,24.0,1,21,1,5,8.0,2.0,0.7516,3,0.4162,1,0,0,0,0,0,0,0,1,0,0,1,0,0


In [ ]:
final_test_s = StandardScaler().fit_transform(to_clean_test)
final_test = to_clean_test.iloc[:]

In [ ]:
xgbreg.predict(final_test_s).mean()

0.1894053

In [ ]:
submission = pd.DataFrame()
submission['Employee_ID'] = test_df['Employee_ID'].iloc[:]
submission['Attrition_rate'] = p
submission

,Employee_ID,Attrition_rate
0,EID_22713,0.191009
1,EID_9658,0.191009
2,EID_22203,0.191009
3,EID_7652,0.191009
4,EID_6516,0.191009
...,...,...
2995,EID_22547,0.191009
2996,EID_10066,0.191009
2997,EID_7126,0.191009
2998,EID_4929,0.191009


In [ ]:
submission.to_csv('Submission.csv', index=False)

In [ ]:
def xgb_evaluate(
          max_depth,
          gamma,
          min_child_weight,
          max_delta_step,
          subsample,
          colsample_bytree
         ):
    params = {
              'booster' : 'gbtree',
              'max_depth' : int(max_depth),
              'gamma' : gamma,
              'eta' : 0.1,
              'nthread' : 4,
              'silent' : True,
              'eval_metric': 'rmse',
              'subsample' : max(min(subsample, 1), 0),
              'colsample_bytree' : max(min(colsample_bytree, 1), 0),
              'min_child_weight' : min_child_weight,
              'max_delta_step' : int(max_delta_step),
              'seed' : 1001
              }
    # Used around 1000 boosting rounds in the full model
    cv_result = xgb.cv(
                    params,
                    dtrain,
                    num_boost_round = 20000,
                    #stratified = True,
                    nfold = 5,
                    metrics='rmse',
                    #verbose_eval = 10,
                    early_stopping_rounds = 100,
                    #show_stdv = True
               )    
    
    # Bayesian optimization only knows how to maximize, not minimize, so return the negative RMSE
    return -1.0 * cv_result['test-rmse-mean'].iloc[-1]
xgb_bo = BayesianOptimization(xgb_evaluate, {
                                     'max_depth': (2, 12),
                                     'gamma': (0.001, 10.0),
                                     'min_child_weight': (0, 20),
                                     'max_delta_step': (0, 10),
                                     'subsample': (0.4, 1.0),
                                     'colsample_bytree' :(0.4, 1.0)
                                    })
# Use the expected improvement acquisition function to handle negative numbers
# Optimally needs quite a few more initiation points and number of iterations
xgb_bo.maximize(init_points=10, n_iter=30, acq='ei')

|   iter    |  target   | colsam... |   gamma   | max_de... | max_depth | min_ch... | subsample |
-------------------------------------------------------------------------------------------------
|  1        | -0.1857   |  0.4818   |  7.359    |  6.179    |  10.75    |  3.836    |  0.8911   |
|  2        | -0.1857   |  0.5533   |  4.867    |  9.907    |  2.14     |  12.76    |  0.8766   |
|  3        | -0.1857   |  0.741    |  3.101    |  3.102    |  4.36     |  18.41    |  0.717    |
|  4        | -0.1857   |  0.8048   |  2.976    |  0.854    |  8.615    |  6.056    |  0.5082   |
|  5        | -0.1862   |  0.6668   |  0.08425  |  6.469    |  3.553    |  13.56    |  0.662    |
|  6        | -0.1857   |  0.7387   |  2.373    |  2.931    |  8.623    |  0.4284   |  0.4885   |
|  7        | -0.1857   |  0.4499   |  2.0      |  3.982    |  7.1      |  2.039    |  0.9743   |
|  8        | -0.1857   |  0.6844   |  3.552    |  4.935    |  3.391    |  5.917    |  0.6488   |
|  9        | -0.185

In [ ]:
params = xgb_bo.max['params']
params['max_depth'] = int(params['max_depth'])
params['min_child_weight'] = int(params['min_child_weight'])
params['max_delta_step'] = int(params['max_delta_step'])
params

{'colsample_bytree': 0.9509107154535124,
 'gamma': 9.705011417590331,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 0,
 'subsample': 0.5119595128928055}

In [ ]:
model = xgb.train(params, dtrain, num_boost_round=30000)

In [ ]:
model.predict(dtrain)

array([0.18866332, 0.18866332, 0.18866332, ..., 0.18866332, 0.18866332,
       0.18866332], dtype=float32)

In [ ]:
from sklearn.metrics import mean_squared_error
1 - np.sqrt(mean_squared_error(final_features['Attrition_rate'], model.predict(dtrain)))

0.814258702721271

In [ ]:
dtest = xgb.DMatrix(final_test)
dtest

In [ ]:
p = model.predict(dtest)
p

array([0.19100863, 0.19100863, 0.19100863, ..., 0.19100863, 0.19100863,
       0.19100863], dtype=float32)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout

nn = Sequential()
nn.add(Dense(32, input_dim=22, activation='elu'))
nn.add(BatchNormalization())
nn.add(Dropout(0.2))

nn.add(Dense(64, activation='relu'))
nn.add(BatchNormalization())
nn.add(Dropout(0.2))
nn.add(Dense(64, activation='relu'))
nn.add(BatchNormalization())
nn.add(Dropout(0.2))
nn.add(Dense(64, activation='relu'))
nn.add(BatchNormalization())
nn.add(Dropout(0.2))
nn.add(Dense(128, activation='relu'))
nn.add(BatchNormalization())
nn.add(Dropout(0.3))
nn.add(Dense(128, activation='relu'))
nn.add(BatchNormalization())
nn.add(Dropout(0.3))
nn.add(Dense(256, activation='relu'))
nn.add(BatchNormalization())
nn.add(Dropout(0.4))
nn.add(Dense(256, activation='relu'))
nn.add(BatchNormalization())
nn.add(Dropout(0.4))
nn.add(Dense(512, activation='relu'))
nn.add(BatchNormalization())
nn.add(Dropout(0.45))
nn.add(Dense(1024, activation='relu'))
nn.add(BatchNormalization())
nn.add(Dropout(0.5))

nn.add(Dense(512, activation='relu'))
nn.add(BatchNormalization())
nn.add(Dropout(0.4))

nn.add(Dense(128, activation='relu'))
nn.add(BatchNormalization())
nn.add(Dropout(0.2))

nn.add(Dense(1, activation='sigmoid'))

In [ ]:
nn.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_94 (Dense)             (None, 32)                736       
_________________________________________________________________
batch_normalization_85 (Batc (None, 32)                128       
_________________________________________________________________
dropout_85 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_95 (Dense)             (None, 64)                2112      
_________________________________________________________________
batch_normalization_86 (Batc (None, 64)                256       
_________________________________________________________________
dropout_86 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_96 (Dense)             (None, 64)              

In [ ]:
nn.compile(optimizer='sgd', loss='mean_squared_error', metrics=['mse'])

In [ ]:
nn.fit(Xts, Y_train,
       batch_size=8, epochs=10,
       validation_data=(xtst, Y_test),
       shuffle=True)

Train on 5950 samples, validate on 1050 samples
Epoch 1/10
5950/5950 [==============================] - 16s 3ms/step - loss: 0.0537 - mse: 0.0537 - val_loss: 0.0365 - val_mse: 0.0365
Epoch 2/10
5950/5950 [==============================] - 15s 2ms/step - loss: 0.0349 - mse: 0.0349 - val_loss: 0.0353 - val_mse: 0.0353
Epoch 3/10
5950/5950 [==============================] - 14s 2ms/step - loss: 0.0345 - mse: 0.0345 - val_loss: 0.0353 - val_mse: 0.0353
Epoch 4/10
5950/5950 [==============================] - 14s 2ms/step - loss: 0.0344 - mse: 0.0344 - val_loss: 0.0353 - val_mse: 0.0353
Epoch 5/10
5950/5950 [==============================] - 14s 2ms/step - loss: 0.0344 - mse: 0.0344 - val_loss: 0.0353 - val_mse: 0.0353
Epoch 6/10
5950/5950 [==============================] - 14s 2ms/step - loss: 0.0344 - mse: 0.0344 - val_loss: 0.0353 - val_mse: 0.0353
Epoch 7/10
5950/5950 [==============================] - 14s 2ms/step - loss: 0.0344 - mse: 0.0344 - val_loss: 0.0353 - val_mse: 0.0353
Epoch 8

In [ ]:
final_features

,Time_of_service,Post_Level,Pay_Scale,Work_Life_balance,VAR2,VAR7,Attrition_rate,Gender_F,Gender_M,Unit_IT,Unit_Operarions,Unit_Purchasing,Decision_skill_possess_Behavioral,Decision_skill_possess_Conceptual,Compensation_and_Benefits_type0,Compensation_and_Benefits_type3
0,4.0,1,7.0,3.0,0.7516,3,0.1841,1,0,1,0,0,0,1,0,0
1,5.0,3,6.0,4.0,-0.9612,3,0.0670,0,1,0,0,0,0,0,0,0
2,27.0,2,8.0,1.0,-0.9612,3,0.0851,1,0,0,0,0,0,1,0,0
3,4.0,3,8.0,1.0,-1.8176,3,0.0668,1,0,0,0,0,1,0,0,0
4,5.0,3,2.0,3.0,0.7516,2,0.1827,1,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,4.0,2,6.0,3.0,0.7516,3,0.7839,1,0,0,1,0,1,0,0,1
6996,16.0,2,4.0,3.0,-0.1048,4,0.1055,1,0,0,0,0,0,0,0,1
6997,10.0,2,3.0,4.0,-0.1048,3,0.7847,1,0,1,0,0,0,0,0,0
6998,24.0,5,8.0,2.0,0.7516,3,0.4162,1,0,0,0,0,1,0,1,0


In [ ]:
nn.predict(final_features.drop(['Attrition_rate'], axis=1)).mean()

0.16868775

In [ ]:
train_nn = pd.concat([numerical_train, categorical_train], axis=1)
train_nn

,Age,Education_Level,Time_of_service,Time_since_promotion,growth_rate,Travel_Rate,Post_Level,Pay_Scale,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,Attrition_rate,Gender,Relationship_Status,Hometown,Unit,Decision_skill_possess,Compensation_and_Benefits
0,42.0,4,4.0,4,33,1,1,7.0,3.0,4,0.7516,1.8688,2.0,4,5,3,0.1841,0,0,1,2,2,2
1,24.0,3,5.0,4,36,0,3,6.0,4.0,3,-0.9612,-0.4537,2.0,3,5,3,0.0670,1,1,3,3,0,2
2,58.0,3,27.0,3,51,0,2,8.0,1.0,4,-0.9612,-0.4537,3.0,3,8,3,0.0851,0,0,0,8,2,2
3,26.0,3,4.0,3,56,1,3,8.0,1.0,3,-1.8176,-0.4537,NaN,3,7,3,0.0668,0,1,2,1,1,2
4,31.0,1,5.0,4,62,1,3,2.0,3.0,1,0.7516,-0.4537,2.0,2,8,2,0.1827,0,0,3,3,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,23.0,5,4.0,1,20,1,2,6.0,3.0,1,0.7516,-0.4537,2.0,3,7,3,0.7839,0,0,1,5,1,3
6996,44.0,1,16.0,2,59,0,2,4.0,3.0,2,-0.1048,-2.7762,2.0,4,8,4,0.1055,0,0,2,9,0,3
6997,49.0,3,10.0,1,59,1,2,3.0,4.0,2,-0.1048,-0.4537,2.0,4,8,3,0.7847,0,1,3,2,3,2
6998,47.0,3,24.0,1,21,1,5,8.0,2.0,4,0.7516,1.8688,2.0,3,6,3,0.4162,0,0,4,10,1,0


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(train_nn.drop(columns=['Attrition_rate']), train_nn['Attrition_rate'],
                                                    test_size=0.15, random_state=42)

In [ ]:
Xts = StandardScaler().fit_transform(X_train)
xtst = StandardScaler().fit_transform(X_test)

In [ ]:
nn.fit(Xts, Y_train,
       batch_size=8, epochs=30,
       validation_data=(xtst, Y_test),
       shuffle=True)

Train on 5950 samples, validate on 1050 samples
Epoch 1/30
5950/5950 [==============================] - 22s 4ms/step - loss: 0.0735 - mse: 0.0735 - val_loss: 0.0393 - val_mse: 0.0393
Epoch 2/30
5950/5950 [==============================] - 19s 3ms/step - loss: 0.0429 - mse: 0.0429 - val_loss: 0.0355 - val_mse: 0.0355
Epoch 3/30
5950/5950 [==============================] - 19s 3ms/step - loss: 0.0384 - mse: 0.0384 - val_loss: 0.0354 - val_mse: 0.0354
Epoch 4/30
5950/5950 [==============================] - 19s 3ms/step - loss: 0.0369 - mse: 0.0369 - val_loss: 0.0362 - val_mse: 0.0362
Epoch 5/30
5950/5950 [==============================] - 19s 3ms/step - loss: 0.0361 - mse: 0.0361 - val_loss: 0.0353 - val_mse: 0.0353
Epoch 6/30
5950/5950 [==============================] - 19s 3ms/step - loss: 0.0357 - mse: 0.0357 - val_loss: 0.0353 - val_mse: 0.0353
Epoch 7/30
5950/5950 [==============================] - 19s 3ms/step - loss: 0.0356 - mse: 0.0356 - val_loss: 0.0355 - val_mse: 0.0355
Epoch 8

In [ ]:
test_nn = pd.concat([num_test, cat_test], axis=1)
test_nn

,Age,Education_Level,Time_of_service,Time_since_promotion,growth_rate,Travel_Rate,Post_Level,Pay_Scale,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,Gender,Relationship_Status,Hometown,Unit,Decision_skill_possess,Compensation_and_Benefits
0,32.0,5,7.0,4,30,1,5,4.0,1.0,3,-0.9612,-0.4537,2.0,1,8,4,0,1,3,9,2,2
1,65.0,2,41.0,2,72,1,1,1.0,1.0,4,-0.9612,0.7075,1.0,2,8,2,1,1,2,2,3,2
2,52.0,3,21.0,3,25,0,1,8.0,1.0,4,-0.1048,0.7075,2.0,1,9,3,1,0,3,10,3,3
3,50.0,5,11.0,4,28,1,1,2.0,4.0,3,-0.1048,0.7075,2.0,2,8,3,1,1,4,4,0,0
4,44.0,3,12.0,4,47,1,3,2.0,4.0,4,1.6081,0.7075,2.0,2,7,4,0,0,1,9,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,32.0,3,3.0,3,38,1,3,6.0,2.0,1,-0.1048,0.7075,1.0,4,7,4,0,1,1,10,3,2
2996,NaN,2,10.0,2,57,1,4,9.0,1.0,3,0.7516,0.7075,3.0,4,8,3,0,1,1,4,2,2
2997,60.0,4,35.0,1,69,0,2,9.0,3.0,3,-1.8176,-0.4537,NaN,4,8,4,0,1,2,3,0,3
2998,51.0,1,23.0,1,30,0,2,6.0,1.0,5,0.7516,-0.4537,2.0,3,9,2,0,0,3,2,1,3


In [ ]:
test_nn_S = StandardScaler().fit_transform(test_nn)
nn.predict(test_nn_S, batch_size=8)

array([[0.1888998],
       [0.1888998],
       [0.1888998],
       ...,
       [0.1888998],
       [0.1888998],
       [0.1888998]], dtype=float32)

In [ ]:
nn.save_weights('first_30.h5', overwrite=True)

In [ ]:
nn.load_weights('first_30.h5')

In [ ]:
nn.predict(test_nn_S, batch_size=16)

array([[0.1888998],
       [0.1888998],
       [0.1888998],
       ...,
       [0.1888998],
       [0.1888998],
       [0.1888998]], dtype=float32)

In [ ]:
nn.fit(Xts, Y_train,
       batch_size=8, epochs=20,
       validation_data=(xtst, Y_test),
       shuffle=True, initial_epoch=5)

Train on 5950 samples, validate on 1050 samples
Epoch 6/20
5950/5950 [==============================] - 15s 3ms/step - loss: 0.0345 - mse: 0.0345 - val_loss: 0.0353 - val_mse: 0.0353
Epoch 7/20
5950/5950 [==============================] - 15s 2ms/step - loss: 0.0345 - mse: 0.0345 - val_loss: 0.0353 - val_mse: 0.0353
Epoch 8/20
5950/5950 [==============================] - 15s 2ms/step - loss: 0.0344 - mse: 0.0344 - val_loss: 0.0353 - val_mse: 0.0353
Epoch 9/20
5950/5950 [==============================] - 15s 2ms/step - loss: 0.0344 - mse: 0.0344 - val_loss: 0.0353 - val_mse: 0.0353
Epoch 10/20
5950/5950 [==============================] - 15s 2ms/step - loss: 0.0345 - mse: 0.0345 - val_loss: 0.0353 - val_mse: 0.0353
Epoch 11/20
5950/5950 [==============================] - 15s 2ms/step - loss: 0.0344 - mse: 0.0344 - val_loss: 0.0353 - val_mse: 0.0353
Epoch 12/20
5950/5950 [==============================] - 15s 2ms/step - loss: 0.0345 - mse: 0.0345 - val_loss: 0.0353 - val_mse: 0.0353
Epoc

In [ ]:
nnn.predict(test_nn_S)

array([[0.1820182],
       [0.1820182],
       [0.1820182],
       ...,
       [0.1820182],
       [0.1820182],
       [0.1820182]], dtype=float32)

In [ ]:
submission = pd.DataFrame()
submission['Employee_ID'] = test_df['Employee_ID'].iloc[:]
submission['Attrition_rate'] = 
submission

,Employee_ID,Attrition_rate
0,EID_22713,0.196985
1,EID_9658,0.196985
2,EID_22203,0.196985
3,EID_7652,0.196985
4,EID_6516,0.196985
...,...,...
2995,EID_22547,0.196985
2996,EID_10066,0.196985
2997,EID_7126,0.196985
2998,EID_4929,0.196985


In [ ]:
submission.to_csv('Submission.csv', index=False)

In [ ]:
nnn = Sequential()
nnn.add(Dense(32, input_dim=22, activation='relu'))
nnn.add(BatchNormalization())
nnn.add(Dropout(0.2))

nnn.add(Dense(64, activation='sigmoid'))
nnn.add(BatchNormalization())
nnn.add(Dropout(0.2))
nnn.add(Dense(64, activation='relu'))
nnn.add(BatchNormalization())
nnn.add(Dropout(0.2))
nnn.add(Dense(128, activation='sigmoid'))
nnn.add(BatchNormalization())
nnn.add(Dropout(0.2))
nnn.add(Dense(128, activation='relu'))
nnn.add(BatchNormalization())
nnn.add(Dropout(0.2))
nnn.add(Dense(256, activation='sigmoid'))
nnn.add(BatchNormalization())
nnn.add(Dropout(0.3))
nnn.add(Dense(256, activation='relu'))
nnn.add(BatchNormalization())
nnn.add(Dropout(0.3))
nnn.add(Dense(512, activation='sigmoid'))
nnn.add(BatchNormalization())
nnn.add(Dropout(0.4))
nnn.add(Dense(1024, activation='relu'))
nnn.add(BatchNormalization())
nnn.add(Dropout(0.5))

nnn.add(Dense(512, activation='relu'))
nnn.add(BatchNormalization())
nnn.add(Dropout(0.2))

nnn.add(Dense(128, activation='sigmoid'))
nnn.add(BatchNormalization())
nnn.add(Dropout(0.2))

nnn.add(Dense(64, activation='relu'))
nnn.add(BatchNormalization())
nnn.add(Dropout(0.2))
nnn.add(Dense(1, activation='sigmoid'))

nnn.compile(loss='mean_squared_error', optimizer='sgd', metrics=['mse'])
nnn.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_146 (Dense)            (None, 32)                736       
_________________________________________________________________
batch_normalization_133 (Bat (None, 32)                128       
_________________________________________________________________
dropout_133 (Dropout)        (None, 32)                0         
_________________________________________________________________
dense_147 (Dense)            (None, 64)                2112      
_________________________________________________________________
batch_normalization_134 (Bat (None, 64)                256       
_________________________________________________________________
dropout_134 (Dropout)        (None, 64)                0         
_________________________________________________________________
dense_148 (Dense)            (None, 64)              

In [ ]:
nnn.fit(Xts, Y_train,
        batch_size=16, epochs=80, initial_epoch=0,
        validation_data=(xtst, Y_test), shuffle=True)

Train on 5950 samples, validate on 1050 samples
Epoch 1/80
5950/5950 [==============================] - 9s 1ms/step - loss: 0.1516 - mse: 0.1516 - val_loss: 0.0614 - val_mse: 0.0614
Epoch 2/80
5950/5950 [==============================] - 7s 1ms/step - loss: 0.0813 - mse: 0.0813 - val_loss: 0.0382 - val_mse: 0.0382
Epoch 3/80
5950/5950 [==============================] - 7s 1ms/step - loss: 0.0600 - mse: 0.0600 - val_loss: 0.0357 - val_mse: 0.0357
Epoch 4/80
5950/5950 [==============================] - 7s 1ms/step - loss: 0.0508 - mse: 0.0508 - val_loss: 0.0354 - val_mse: 0.0354
Epoch 5/80
5950/5950 [==============================] - 7s 1ms/step - loss: 0.0482 - mse: 0.0482 - val_loss: 0.0353 - val_mse: 0.0353
Epoch 6/80
5950/5950 [==============================] - 7s 1ms/step - loss: 0.0447 - mse: 0.0447 - val_loss: 0.0353 - val_mse: 0.0353
Epoch 7/80
5950/5950 [==============================] - 7s 1ms/step - loss: 0.0433 - mse: 0.0433 - val_loss: 0.0355 - val_mse: 0.0355
Epoch 8/80
595

In [ ]:
from sklearn.preprocessing import LabelEncoder

for col in cat_test.columns:
  cat_test[col] = LabelEncoder().fit_transform(cat_test[col])
cat_test

,Gender,Relationship_Status,Hometown,Unit,Decision_skill_possess,Compensation_and_Benefits
0,0,1,3,9,2,2
1,1,1,2,2,3,2
2,1,0,3,10,3,3
3,1,1,4,4,0,0
4,0,0,1,9,2,2
...,...,...,...,...,...,...
2995,0,1,1,10,3,2
2996,0,1,1,4,2,2
2997,0,1,2,3,0,3
2998,0,0,3,2,1,3
